<a href="https://colab.research.google.com/github/yuki-tamaribuchi/ml_code_kata/blob/master/mnist_subclassing_fully_connected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

x_train,x_test=np.divide(x_train.astype('float32'),255.0),np.divide(x_test.astype('float32'),255.0)
x_train,x_test=x_train.reshape(60000,28,28,1),x_test.reshape(10000,28,28,1)
y_train,y_test=np.array(tf.one_hot(y_train,10)),np.array(tf.one_hot(y_test,10))

In [3]:
class SubClassing(tf.keras.Model):
  def __init__(self):
    super(SubClassing,self).__init__()
    self.flatten=tf.keras.layers.Flatten()
    self.hidden=tf.keras.layers.Dense(128,activation='relu')
    self.outputs=tf.keras.layers.Dense(10,activation='softmax')

  def call(self,x):
    x=self.flatten(x)
    x=self.hidden(x)
    return self.outputs(x)

In [4]:
@tf.function
def train_step(images,labels):
  with tf.GradientTape() as tape:
    logits=model(images)

    loss_value=loss_object(y_true=labels,y_pred=logits)

  grads=tape.gradient(loss_value,model.trainable_variables)
  optimizer.apply_gradients(zip(grads,model.trainable_variables))
  train_loss(loss_value)
  train_accuracy(y_true=labels,y_pred=logits)

In [5]:
@tf.function
def test_step(images,labels):
  predictions=model(images)
  t_loss=loss_object(labels,predictions)
  test_loss(t_loss)
  test_accuracy(labels,predictions)

In [6]:
loss_object=tf.keras.losses.CategoricalCrossentropy()
optimizer=tf.keras.optimizers.Adam()
train_loss=tf.keras.metrics.Mean(name='train_loss')
train_accuracy=tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
test_loss=tf.keras.metrics.Mean(name='test_loss')
test_accuracy=tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [7]:
y_train[0:3]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

In [8]:
def train(model,x_train,t_train,epochs,batch_size):
  ds_train=tf.data.Dataset.from_tensor_slices((x_train,t_train)).shuffle(10000).batch(32)

  for epoch in range(epochs):
    for ds in ds_train:
      train_step(ds[0],ds[1])

    template='Epoch {}, Loss:{}, Accuracy:{}'
    tf.print(template.format(epoch+1,
                             train_loss.result(),
                             train_accuracy.result()*100
                             ))
  train_loss.reset_states()
  train_accuracy.reset_states()


model=SubClassing()
train(model,x_train,y_train,5,32)

Epoch 1, Loss:0.2679767310619354, Accuracy:92.33000183105469
Epoch 2, Loss:0.19234006106853485, Accuracy:94.48583221435547
Epoch 3, Loss:0.15503542125225067, Accuracy:95.50333404541016
Epoch 4, Loss:0.13106630742549896, Accuracy:96.16542053222656
Epoch 5, Loss:0.11420044302940369, Accuracy:96.63866424560547


In [9]:
ds_test=tf.data.Dataset.from_tensor_slices((x_test,y_test)).shuffle(10000).batch(32)
for ds in ds_test:
  test_step(ds[0],ds[1])
template='Loss:{}, Accuracy:{}'
print(template.format(test_loss.result(),test_accuracy.result()*100))

Loss:0.07553506642580032, Accuracy:97.64999389648438
